<a href="https://colab.research.google.com/github/wheezyturbo/illeterate-computing-machine/blob/main/marksScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os


def rrmdir(path):
    for entry in os.scandir(path):
        if entry.is_dir():
            rrmdir(entry)
        else:
            os.remove(entry)
    os.rmdir(path)
rrmdir('/content/illeterate-computing-machine')
os.chdir('/content/')

In [19]:
!ls
!git clone https://github.com/wheezyturbo/illeterate-computing-machine.git

sample_data
Cloning into 'illeterate-computing-machine'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 35 (delta 9), reused 28 (delta 4), pack-reused 0
Unpacking objects: 100% (35/35), 4.19 MiB | 5.95 MiB/s, done.


In [20]:
cd illeterate-computing-machine/


/content/illeterate-computing-machine


In [ ]:
!apt install firefox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libdbus-glib-1-2 libdbusmenu-glib4 libdbusmenu-gtk3-4 xul-ext-ubufox
Suggested packages:
  fonts-lyx
The following NEW packages will be installed:
  firefox libdbus-glib-1-2 libdbusmenu-glib4 libdbusmenu-gtk3-4 xul-ext-ubufox
0 upgraded, 5 newly installed, 0 to remove and 22 not upgraded.
Need to get 60.4 MB of archives.
After this operation, 241 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libdbus-glib-1-2 amd64 0.110-5fakssync1 [59.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 firefox amd64 110.0.1+build2-0ubuntu0.20.04.1 [60.2 MB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libdbusmenu-glib4 amd64 16.04.1+18.10.20180917-0ubuntu6 [41.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libdbusmenu-gtk3-4 amd64 16.04.1+18.10.20180917-0ubunt

In [21]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **this is without flask it will print the scrapped data into the console ie the marks **: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify,render_template,request

# app = Flask(__name__)


regarray = ['mm20ccsr22','mm20ccsr18', 'mm20ccsr16', 'mm20ccsr13','mm20ccsr14','mm20ccsr19','mm20ccsr30','WM20BCAR02','WM20BCAR06','WM20BCAR12','WM20BCAR01','WM20BCAR03']
aadhararray = [686185381631,578533381676, 823508626405, 783917821528,292531761206,780080049949,885417748021,331693014683,250826071094,632336907426,814114687132,373297720804]
# regarray = ['WM20BCAR02']
# aadhararray = [331693014683]


# Set Firefox options to run in headless mode
options = Options()
options.add_argument('-headless')

cs = []

def scrape(link):
      cs.clear()
      driver = webdriver.Firefox(options = options)
      for i in range(len(aadhararray)):
      # Create Firefox instance with headless option
            # driver.get('http://www.exam.kannuruniversity.ac.in/UG/bsc5semresult2021/result19.php')
            # driver.get('http://www.exam.kannuruniversity.ac.in/UG/bsc4semresult2022_new/result19.php')
            driver.get(link)

            regno_ = driver.find_element("xpath", '//*[@id="regno"]')
            regno_.send_keys(regarray[i])

            aadhar = driver.find_element("xpath", '//*[@id="aadhaar"]')
            aadhar.send_keys(int(aadhararray[i]))
            
            driver.find_element("xpath",'//*[@id="cut"]')\
                  .click()
            
            # Wait for the result page to load
            wait = WebDriverWait(driver, 10)  # maximum wait time of 10 seconds
            wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]')))
            
            name_ = driver.find_element("xpath", '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]').text
            # markp = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%; top: 55.86%")]').text
            


            #scraping the percentage
            # markp = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%")]').text
            markp_element = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%") or contains(@style, "left: 36.17%")]')
            if markp_element.text != '-':
                  markp = markp_element.text
            else:
                  markp = 'Fail'


            
            print(name_," completed...")

            # scraping marks
            # mark_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 71.38%")]')
            # marks = [mark_element.text for mark_element in mark_elements]
            mark_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 71.38%")]')
            marks = []
            for mark_element in mark_elements:
                  if mark_element.text != '-':
                        marks.append(mark_element.text)
                  else:
                        marks.append('Fail')

            subject_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 16.63%")]')
            subjects = [subject_element.text for subject_element in subject_elements]


            mark_dict = {}
            for j in range(len(subjects)):
                  try:
                        mark_dict[subjects[j]] = marks[j]
                  except:
                        mark_dict[subjects[j]] = '0'


            result = {"regno": regarray[i], "name": name_, "percentage": markp, "marks": mark_dict}
            #     print(result)
            cs.append(result)


      driver.quit()
      cs.sort(key=lambda x: float(x['percentage'].replace('Fail', '0')), reverse=True)

      return cs
scrape('http://www.exam.kannuruniversity.ac.in/UG/bsc5semresult2021/result19.php')
print(cs)
# @app.route('/cs')
# def get_cs():
#     link = request.args.get('link')
#     return jsonify(scrape(link))

# @app.route('/')
# def index():
    
#     return render_template("test.html")

# if __name__ == '__main__':
#     app.run(debug=True)

SANOOP P J  completed...
JOYAL JOSE  completed...
JAIS BINOY  completed...
ATHUL VARKEY  completed...
BIMAL KRISHNA  completed...
LIJO N S  completed...
SHIHANA V P  completed...
EBIN THOMAS  completed...
JESNI FAISAL K F  completed...
RISHWAL T P  completed...
ABHAY FRANCIS  completed...
MUHAMMAD MUBASHEER A  completed...
[{'regno': 'mm20ccsr18', 'name': 'JOYAL JOSE', 'percentage': '83.53', 'marks': {'Web Technology': '38', 'Java Programming': '42', 'Computation Using Python': '38', 'Algorithm Designing': '44', 'Quantitative Arithmetic and Reasoning': '25'}}, {'regno': 'mm20ccsr22', 'name': 'SANOOP P J', 'percentage': '83.41', 'marks': {'Web Technology': '44', 'Java Programming': '41', 'Computation Using Python': '35', 'Algorithm Designing': '41', 'Quantitative Arithmetic and Reasoning': '25'}}, {'regno': 'mm20ccsr16', 'name': 'JAIS BINOY', 'percentage': '82.94', 'marks': {'Web Technology': '43', 'Java Programming': '40', 'Computation Using Python': '39', 'Algorithm Designing': '40', 

In [ ]:
%%sh
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.2M  100 13.2M    0     0  13.4M      0 --:--:-- --:--:-- --:--:-- 13.4M


In [ ]:
!cp ngrok /usr/bin/
!ngrok authtoken 2Mnv3hTNbuOWJqgO0sdW075jkH8_FfZe3pv1YoK43CdtGw6X


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


**click the running on ngrok.io link to access the dummy front end that uses the api to display the marks in the form of cards also wait for after clikcing the do it button because this takes some time to scrape and show the results**

In [12]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify,render_template,request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 

regarray = ['mm20ccsr22','mm20ccsr18', 'mm20ccsr16', 'mm20ccsr13','mm20ccsr14','mm20ccsr19','mm20ccsr30','WM20BCAR02','WM20BCAR06','WM20BCAR12','WM20BCAR01','WM20BCAR03']
aadhararray = [686185381631,578533381676, 823508626405, 783917821528,292531761206,780080049949,885417748021,331693014683,250826071094,632336907426,814114687132,373297720804]
# regarray = ['WM20BCAR02']
# aadhararray = [331693014683]


# Set Firefox options to run in headless mode
options = Options()
options.add_argument('-headless')

cs = []

def scrape(link):
      cs.clear()
      driver = webdriver.Firefox(options = options)
      for i in range(len(aadhararray)):
        try:
      # Create Firefox instance with headless option
            # driver.get('http://www.exam.kannuruniversity.ac.in/UG/bsc5semresult2021/result19.php')
            # driver.get('http://www.exam.kannuruniversity.ac.in/UG/bsc4semresult2022_new/result19.php')
            driver.get(link)

            regno_ = driver.find_element("xpath", '//*[@id="regno"]')
            regno_.send_keys(regarray[i])

            aadhar = driver.find_element("xpath", '//*[@id="aadhaar"]')
            aadhar.send_keys(int(aadhararray[i]))
            
            driver.find_element("xpath",'//*[@id="cut"]')\
                  .click()
            
            # Wait for the result page to load
            wait = WebDriverWait(driver, 5)  # maximum wait time of 10 seconds
            wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]')))
            
            name_ = driver.find_element("xpath", '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]').text
            # markp = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%; top: 55.86%")]').text
            


            #scraping the percentage
            # markp = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%")]').text
            markp_element = driver.find_element("xpath",'//span[contains(@style, "left: 34.35%") or contains(@style, "left: 36.17%")]')
            if markp_element.text != '-':
                  markp = markp_element.text
            else:
                  markp = 'Fail'


            
            print(name_," completed...")

            # scraping marks
            # mark_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 71.38%")]')
            # marks = [mark_element.text for mark_element in mark_elements]
            mark_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 71.38%")]')
            marks = []
            for mark_element in mark_elements:
                  if mark_element.text != '-':
                        marks.append(mark_element.text)
                  else:
                        marks.append('Fail')

            subject_elements = driver.find_elements("xpath", '//span[contains(@style, "left: 16.63%")]')
            subjects = [subject_element.text for subject_element in subject_elements]


            mark_dict = {}
            for j in range(len(subjects)):
                  try:
                        mark_dict[subjects[j]] = marks[j]
                  except:
                        mark_dict[subjects[j]] = '0'


            result = {"regno": regarray[i], "name": name_, "percentage": markp, "marks": mark_dict}
            #     print(result)
            cs.append(result)
        except Exception as e:
          print(regarray[i] ,'exception' , e)

      driver.quit()
      try:
        cs.sort(key=lambda x: float(x['percentage'].replace('Fail'or'-', '0')), reverse=True)
      except:
        print('replace exception')

      return cs
@app.route('/cs')
def get_cs():
    link = request.args.get('link')
    return jsonify(scrape(link))

@app.route('/')
def index():
    
    return render_template("test.html")

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6800-35-203-158-22.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify,render_template,request
import bca
import bsccs
import re

def get_course_semester(url):
    # Extract the course and semester information from the URL using regex
    match = re.search(r'/(\w+\d+)semresult', url)
    if match:
        course_semester = match.group(1)
        course = re.search(r'^\D+', course_semester).group()
        semester = int(re.search(r'\d+$', course_semester).group())
        print(semester)
        return (course, semester)
    else:
        return None

# from flask_ngrok import run_with_ngrok

app = Flask(__name__)
# run_with_ngrok(app) 

regarray = ['mm20ccsr22','mm20ccsr18', 'mm20ccsr16', 'mm20ccsr13','mm20ccsr14','mm20ccsr19','mm20ccsr30','WM20BCAR02','WM20BCAR06','WM20BCAR12','WM20BCAR01','WM20BCAR03']
aadhararray = [686185381631,578533381676, 823508626405, 783917821528,292531761206,780080049949,885417748021,331693014683,250826071094,632336907426,814114687132,373297720804]
# regarray = ['WM20BCAR02']
# aadhararray = [331693014683]


# Set Firefox options to run in headless mode
options = Options()
options.add_argument('-headless')

cs = []

def scrape(link):
    print("starting...")
    course, semester = get_course_semester(link)
    print(course,semester)
    cs.clear()
    marks_dict = {}
    driver = webdriver.Firefox(options=options)
    for i in range(len(aadhararray)):
        try:
            driver.get(link)
            course_names = []
            if 'ccsr' in regarray[i].lower():
                course_names = bsccs.course_names(semester)
            elif 'bcar' in regarray[i].lower():
                course_names = bca.course_names(semester)
            regno_ = driver.find_element("xpath", '//*[@id="regno"]')
            regno_.send_keys(regarray[i])
            aadhar = driver.find_element("xpath", '//*[@id="aadhaar"]')
            aadhar.send_keys(int(aadhararray[i]))
            driver.find_element("xpath", '//*[@id="cut"]') \
                .click()

            wait = WebDriverWait(driver, 5)
            wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]')))

            try:
                course_spans = driver.find_elements(By.TAG_NAME,"span")
                marks_dict = {}
                for course_name in course_names:
                    for span in course_spans:
                        if course_name in span.text:
                            marks = span.find_element('xpath',"./following-sibling::span[10]").text
                            marks_dict[course_name] = marks
                            break

                print(marks_dict)
            except Exception as e:
                print(e)

            name_ = driver.find_element("xpath", '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]').text
            markp_element = driver.find_element("xpath", '//span[contains(@style, "left: 34.35%") or contains(@style, "left: 36.17%")]')
            if markp_element.text != '-':
                markp = markp_element.text
            else:
                markp = 'Fail'

            print(name_, " completed...")

            result = {"regno": regarray[i], "name": name_, "percentage": markp, "marks": marks_dict}
            cs.append(result)
        except:
            print(regarray[i], 'exception')

    driver.quit()
    try:
        cs.sort(key=lambda x: float(x['percentage'].replace('Fail' or '-', '0')), reverse=True)
    except:
        print('replace exception')

    return cs





# print(scrape('http://www.exam.kannuruniversity.ac.in/UG/bsc2semresult2020_new/result19.php'))

@app.route('/cs')
def get_cs():
    link = request.args.get('link')
    return jsonify(scrape(link))

@app.route('/')
def index():
    print('req')
    return render_template("test.html")

if __name__ == '__main__':
    app.run(debug=True)


***better Stuff need subjects list predefined in the name modules***

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify,render_template,request
import bca
import bsccs
import re

def get_course_semester(url):
    # Extract the course and semester information from the URL using regex
    match = re.search(r'/(\w+\d+)semresult', url)
    if match:
        course_semester = match.group(1)
        course = re.search(r'^\D+', course_semester).group()
        semester = int(re.search(r'\d+$', course_semester).group())
        print(semester)
        return (course, semester)
    else:
        return None

# from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 

regarray = ['mm20ccsr22','mm20ccsr18', 'mm20ccsr16', 'mm20ccsr13','mm20ccsr14','mm20ccsr19','mm20ccsr30','WM20BCAR02','WM20BCAR06','WM20BCAR12','WM20BCAR01','WM20BCAR03']
aadhararray = [686185381631,578533381676, 823508626405, 783917821528,292531761206,780080049949,885417748021,331693014683,250826071094,632336907426,814114687132,373297720804]
# regarray = ['WM20BCAR02']
# aadhararray = [331693014683]


# Set Firefox options to run in headless mode
options = Options()
options.add_argument('-headless')

cs = []

def scrape(link):
    print("starting...")
    course, semester = get_course_semester(link)
    print(course,semester)
    cs.clear()
    marks_dict = {}
    driver = webdriver.Firefox(options=options)
    for i in range(len(aadhararray)):
        try:
            driver.get(link)
            course_names = []
            if 'ccsr' in regarray[i].lower():
                course_names = bsccs.course_names(semester)
            elif 'bcar' in regarray[i].lower():
                course_names = bca.course_names(semester)
            regno_ = driver.find_element("xpath", '//*[@id="regno"]')
            regno_.send_keys(regarray[i])
            aadhar = driver.find_element("xpath", '//*[@id="aadhaar"]')
            aadhar.send_keys(int(aadhararray[i]))
            driver.find_element("xpath", '//*[@id="cut"]') \
                .click()

            wait = WebDriverWait(driver, 5)
            wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]')))

            try:
                course_spans = driver.find_elements(By.TAG_NAME,"span")
                marks_dict = {}
                for course_name in course_names:
                    for span in course_spans:
                        if course_name in span.text:
                            marks = span.find_element('xpath',"./following-sibling::span[10]").text
                            marks_dict[course_name] = marks
                            break

                print(marks_dict)
            except Exception as e:
                print(e)

            name_ = driver.find_element("xpath", '//span[contains(@style, "left: 14") and contains(@style, "top: 31")]').text
            markp_element = driver.find_element("xpath", '//span[contains(@style, "left: 34.35%") or contains(@style, "left: 36.17%")]')
            if markp_element.text != '-':
                markp = markp_element.text
            else:
                markp = 'Fail'

            print(name_, " completed...")

            result = {"regno": regarray[i], "name": name_, "percentage": markp, "marks": marks_dict}
            cs.append(result)
        except:
            print(regarray[i], 'exception')

    driver.quit()
    try:
        cs.sort(key=lambda x: float(x['percentage'].replace('Fail' or '-', '0')), reverse=True)
    except:
        print('replace exception')

    return cs





# print(scrape('http://www.exam.kannuruniversity.ac.in/UG/bsc2semresult2020_new/result19.php'))

@app.route('/cs')
def get_cs():
    link = request.args.get('link')
    return jsonify(scrape(link))

@app.route('/')
def index():
    print('req')
    return render_template("test.html")

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://deed-35-203-158-22.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Mar/2023 06:53:49] "GET / HTTP/1.1" 200 -


req
starting...
4
bsc 4
{'Digital Electronics': '45', 'Operating Systems': '48', 'Software Engineering': '38', 'LabII: Data Structures Using C++': '25', 'Lab III: Database Managenent System': '24', 'Mathematics for Computer Science IV': '47', 'Statistical Inference': '47'}
SANOOP P J  completed...
{'Digital Electronics': '35', 'Operating Systems': '43', 'Software Engineering': '34', 'LabII: Data Structures Using C++': '25', 'Lab III: Database Managenent System': '24', 'Mathematics for Computer Science IV': '41', 'Statistical Inference': '24'}
JOYAL JOSE  completed...
{'Digital Electronics': '35', 'Operating Systems': '42', 'Software Engineering': '40', 'LabII: Data Structures Using C++': '25', 'Lab III: Database Managenent System': '24', 'Mathematics for Computer Science IV': '47', 'Statistical Inference': '35'}
JAIS BINOY  completed...
{'Digital Electronics': '34', 'Operating Systems': '39', 'Software Engineering': '39', 'LabII: Data Structures Using C++': '21', 'Lab III: Database Man

INFO:werkzeug:127.0.0.1 - - [11/Mar/2023 06:55:48] "GET /cs?link=http://www.exam.kannuruniversity.ac.in/UG/bsc4semresult2022_new/result19.php HTTP/1.1" 200 -


starting...
5
bsc 5
{'Web Technology': '44', 'Java Programming': '41', 'Computation Using Python': '35', 'Algorithm Designing': '41', 'Quantitative Arithmetic and Reasoning': '25'}
SANOOP P J  completed...
{'Web Technology': '38', 'Java Programming': '42', 'Computation Using Python': '38', 'Algorithm Designing': '44', 'Quantitative Arithmetic and Reasoning': '25'}
JOYAL JOSE  completed...
{'Web Technology': '43', 'Java Programming': '40', 'Computation Using Python': '39', 'Algorithm Designing': '40'}
JAIS BINOY  completed...
{'Web Technology': '40', 'Java Programming': '35', 'Computation Using Python': '37', 'Algorithm Designing': '48', 'Quantitative Arithmetic and Reasoning': '25'}
ATHUL VARKEY  completed...
{'Web Technology': '24', 'Java Programming': '29', 'Computation Using Python': '26', 'Algorithm Designing': '28'}
BIMAL KRISHNA  completed...
{'Web Technology': '38', 'Java Programming': '35', 'Computation Using Python': '33', 'Algorithm Designing': '46'}
LIJO N S  completed...
{'

INFO:werkzeug:127.0.0.1 - - [11/Mar/2023 07:07:35] "GET /cs?link=http://www.exam.kannuruniversity.ac.in/UG/bsc5semresult2021/result19.php HTTP/1.1" 200 -


In [23]:
!python /content/illeterate-computing-machine/test_socket.py

 * Serving Flask app 'test_socket'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 558-364-090
